In [1]:
import tensorflow as tf

if tf.test.gpu_device_name():
    print(f"GPU is available: {tf.test.gpu_device_name()}")
else:
    print("GPU not available.")

GPU not available.


## 3. Загрузка и предобработка графа

### Конвертация OWL в формат триплетов
Используем библиотеку rdflib для извлечения триплетов.

In [2]:
from rdflib import Graph

# Загрузка графа OWL
graph = Graph()
graph.parse("skyrim_knowledge_graph.owl", format="xml")

# Извлечение триплетов
triplets = []
for s, p, o in graph:
    triplets.append((str(s), str(p), str(o)))

# Сохранение триплетов в формате CSV
import pandas as pd

triplets_df = pd.DataFrame(triplets, columns=["subject", "predicate", "object"])
triplets_df.to_csv("skyrim_triplets.csv", index=False)

print(f"Extracted {len(triplets)} triplets.")

Extracted 17634 triplets.


## 4. Разбиение данных
### 4.1 Подготовка данных
Загрузим данные и удалим строки с отсутствующими значениями.

In [3]:
# Загрузка триплетов
triplets = pd.read_csv("skyrim_triplets.csv")

# Удаление null значений
triplets.dropna(inplace=True)

In [4]:
triplets

,subject,predicate,object
0,http://example.org/skyrim#Amaund_Motierre,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org/skyrim#Character
1,http://example.org/skyrim#Sifnar_Ironkettle,http://example.org/skyrim#hasSkill,http://example.org/skyrim#Archery
2,http://example.org/skyrim#Oslaf,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org/skyrim#Character
3,http://example.org/skyrim#Vigilant_Tolan,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#NamedIndividual
4,http://example.org/skyrim#Edla,http://example.org/skyrim#isMemberOf,http://example.org/skyrim#DLC2SkaalMerchant
...,...,...,...
17629,http://example.org/skyrim#Anoriath,http://example.org/skyrim#isMemberOf,http://example.org/skyrim#FavorExcludedFaction
17630,http://example.org/skyrim#DLC2SVWulfWildbloods...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#NamedIndividual
17631,http://example.org/skyrim#Black_Book:_The_Hidd...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.w3.org/2002/07/owl#NamedIndividual
17632,http://example.org/skyrim#Moth_gro-Bagol,http://example.org/skyrim#isMemberOf,"http://example.org/skyrim#During_sieges,_staff..."


## Обучение модели

In [18]:
import numpy as np
from ampligraph.latent_features import ScoringBasedEmbeddingModel
from ampligraph.evaluation import train_test_split_no_unseen
from ampligraph.latent_features.loss_functions import get as get_loss
from ampligraph.latent_features.regularizers import get as get_regularizer

In [20]:
# Разделение данных для обучения и валидации
train_triplets, valid = train_test_split_no_unseen(triplets.values, test_size=0.2)

print(f"Training triplets: {len(train_triplets)}, Testing triplets: {len(valid)}")

# Инициализация модели
model = ScoringBasedEmbeddingModel(scoring_type='ComplEx',  k=500, 
               eta=10, seed=32)

# Optimizer, loss and regularizer definition
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
loss = get_loss('multiclass_nll')
regularizer = get_regularizer('LP', {'p': 3, 'lambda': 1e-5})

model.compile(optimizer=optimizer, loss=loss, entity_relation_regularizer=regularizer)
# Обучение модели
model.fit(train_triplets,epochs=400,verbose=True)

Training triplets: 14067, Testing triplets: 3516
Epoch 1/400
16/16 [==============================] - 4s 271ms/step - loss: 2248.7542
Epoch 2/400
16/16 [==============================] - 4s 253ms/step - loss: 2248.6299
Epoch 3/400
16/16 [==============================] - 4s 254ms/step - loss: 2248.5193
Epoch 4/400
16/16 [==============================] - 4s 257ms/step - loss: 2248.4099
Epoch 5/400
16/16 [==============================] - 4s 258ms/step - loss: 2248.2976
Epoch 6/400
16/16 [==============================] - 4s 254ms/step - loss: 2248.1824
Epoch 7/400
16/16 [==============================] - 4s 259ms/step - loss: 2248.0623
Epoch 8/400
16/16 [==============================] - 4s 256ms/step - loss: 2247.9360
Epoch 9/400
16/16 [==============================] - 4s 255ms/step - loss: 2247.8013
Epoch 10/400
16/16 [==============================] - 4s 262ms/step - loss: 2247.6575
Epoch 11/400
16/16 [==============================] - 4s 260ms/step - loss: 2247.5032
Epoch 12/400
1

In [21]:
from ampligraph.evaluation import train_test_split_no_unseen

# Оценка качества
ranks = model.evaluate(valid, verbose=True, use_filter={'train': train_triplets,
                                                        'test': valid})
print("Ранговые метрики:", ranks)

353/353 [==============================] - 38s 107ms/step
Ранговые метрики: [[   4   44]
 [  70 3588]
 [ 531   42]
 ...
 [ 118    8]
 [  26    1]
 [  14    1]]


In [22]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

mr = mr_score(ranks)
mrr = mrr_score(ranks)

print("MRR: %.2f" % (mrr))
print("MR: %.2f" % (mr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

MRR: 0.46
MR: 331.04
Hits@10: 0.61
Hits@3: 0.49
Hits@1: 0.38


## Classification

### Задача классификации:

Цель задачи: Предсказать локацию персонажа в Skyrim.
Признак для классификации: Предсказание локации персонажа по его эмбеддингу в графе.
Тип задачи: Многоклассовая классификация (каждый класс — это одна уникальная локация).
Сколько классов: Количество уникальных локаций (например, 10-20 локаций). Для проверки можно вывести распределение классов.

### Для чего решаем эту задачу:

Цель этой задачи — предсказать, в какой локации находится персонаж на основе его связи с другими сущностями в графе. Это может быть полезно, например, в игре, для оптимизации поиска информации о локациях или для анализа поведения персонажей в разных частях игры. Кроме того некоторые персонажи дают полезные уникальные предметы или навыки и найти их не всегда легко. Информацию о местоположении персонажа не всегда указана даже на вики.

Определение принадлежности к локации:

Признак: http://example.org/skyrim#locatedAt.
Цель: Предсказать локацию персонажа, основываясь на его связях в графе.

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [42]:
# Получение эмбеддингов
characters = [row[0] for row in train_triplets if row[1] == 'http://example.org/skyrim#locatedAt']
locations = list(set([row[2] for row in train_triplets if row[1] == 'http://example.org/skyrim#locatedAt']))

In [43]:
character_embeddings = model.get_embeddings(characters, embedding_type='e')
labels = [locations.index(loc) for loc in [row[2] for row in train_triplets if row[1] == 'http://example.org/skyrim#locatedAt']]

In [47]:
# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(character_embeddings, labels, test_size=0.2)

# Обучение классификатора
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Оценка
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           2       1.00      1.00      1.00         1
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         2
           6       1.00      1.00      1.00        12
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00        10
          11       1.00      1.00      1.00         3
          12       0.83      1.00      0.91         5
          14       1.00      1.00      1.00        10
          17       1.00      1.00      1.00        11
          18       1.00      1.00      1.00         1
          19       0.50      1.00      0.67         1
          20       1.00      1.00      1.00         5
          21       1.00      1.00      1.00         1
          23       1.00      1.00      1.00        12
          24       1.00    

e:\KNOWLEGE_GR\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\KNOWLEGE_GR\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
e:\KNOWLEGE_GR\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
